In [8]:
from lasagne.layers import *  
from lasagne.objectives import *

In [4]:
#FCN used softmax w/ xent, but they also tried sigmoid w/ xent and 
# did just pretty well
def downsample_mask(gt_mask, num_decimation_layers):
    '''gt_mask (Theano Tensor4) -> 768 by 1162 by num_classes
       num_decimation_layer (int) -> number of stride 2 or max pool layers'''
        net = InputLayer(gt_mask, shape=tuple(4*[None]))
        for i in range(num_decimation_layers):
            net = MaxPool2DLayer(net)
        return get_output(net, gt_mask)
            

def get_classwise_scores(last_enc_layer, num_classes):
    '''last_enc_layer  (Theano Tensor4) -> last encoder layer
       num_classes (int) -> should be equal to depth of gt_mask'''
    
    #get hyperplanes for each class
    net = Conv2DLayer(last_enc_layer,
                      num_filters=num_classes,
                      filter_size=1,
                      nonlinearity=)
    return get_output(net)
    
    
def calc_seg_loss(gt_coarse_mask, class_scores):
    '''gt_coarse_mask (Theano Tensor4) -> 5 by 11 by num_classes or something'''
    '''class_scores (Theano Tensor4) -> same as above'''
    
    pass

IndentationError: unexpected indent (<ipython-input-4-e56196b95dc3>, line 4)

In [6]:
Conv2DLayer?

In [7]:
get_output?